In [159]:
%%javascript

IPython.keyboard_manager.command_shortcuts.add_shortcut('Ctrl-k', {
    help : 'move up selected cells',
    help_index : 'jupyter-notebook:move-selection-up',
    handler : function (event) {
        IPython.notebook.move_selection_up();
        return false;
    }}
);

IPython.keyboard_manager.command_shortcuts.add_shortcut('Ctrl-j', {
    help : 'move down selected cells',
    help_index : 'jupyter-notebook:move-selection-down',
    handler :  function (event) {
        IPython.notebook.move_selection_down();
        return false;
    }}
);

<IPython.core.display.Javascript object>

# Feature documentation

## Features Actuales

   ###  Model CountVector: 
    Utilice CountVectorizer de sklearn y usando random forest me quede con los 10 features mas importantes
    
   ### Feature Most Viewed and Searched 
    Me quede con el numero de vistos (en este caso tuve encuenta el modelo) y buscado mas grande en un dia 
    
   ### Lead and conversion
    Cantidad de lead eventos que realizo y cantidad de conversions que realizo
    
   ### Most common searched values in conversion persons count vector
    Las busquedas mas comunes usando countVector()
    
   ### Diff caro
       Magia negra
    
## Features probados que no funcionaron y otras cosas que tampoco:

   ### Cantidad de eventos por tipo
       Numero de veces que realizo cada evento en dataframe
   ### Algoritmo 0.63 con RandomFores
        Use random forest para ver la importancia de lo features de las busquedas y de los modelos y me quede con los mejores 10, el resultado fue menor con 0.61
   
## Best Score: 0.63897 
     - commit 0.63 
     - features 
         - model countvector (sin random forest seleccionando un top)
         - most viewed and searched
         - lead and conversion
         - most common search (sin random forest seleccionando un top)
         - diff caro
               
## Cosas a probar:
    -Kaggle score de lo hecho con random forest
    -
 
 
 
## Acotaciones
    -random forest en search y model, lo que hice fue tomar los features por separado y ver sus importancias
               
        
     

In [9]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from scipy.sparse import hstack
from xgboost.sklearn import XGBClassifier # <3
from sklearn.model_selection import train_test_split
import gc
import matplotlib.pyplot as plt

from scipy.sparse.csr import csr_matrix

import datetime as dt

pd.options.mode.chained_assignment = None


In [2]:
df = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)


In [3]:
df_labels_one = df_labels.loc[df_labels['label']==1]
df_labels_one = df.merge(df_labels_one, on='person', how='right')

In [4]:
subjects = df['person'].drop_duplicates().to_frame()
subjects.shape

(38829, 1)

In [5]:
df['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

In [16]:
df['timestamp']=pd.to_datetime(df['timestamp'])

In [17]:
df_month = df.loc[df['timestamp'].dt.month == 5]

## Modelo count vector

TOP 10 FEATURES ['5s', '6s', 'edge', 'galaxy', 'iphone', 'plus', 's7', 's8', 'samsung', 'se']


try : list of values that exist in conversion

In [21]:
marcas = ['iphone' , 'samsung', 'lg', 'motorola', 'sony', 'asus', 'lenovo']

In [6]:
#df_all = df_labels.merge(df_model, left_on = 'person', right_on = 'person', how = 'left')
#df_all.head()

In [18]:
df_model = df_month.loc[df['event'] == 'viewed product']
df_model = df_model.groupby('person')['model'].apply(list).reset_index()
df_model.head()

,person,model
0,00091926,"[iPhone 6 Plus, iPhone 6S, iPhone 6S, Motorola..."
1,000ba417,"[Samsung Galaxy A3 2016, Samsung Galaxy Gran P..."
2,000c79fe,"[iPhone 7, iPhone 7, iPhone 7]"
3,000e4d9e,"[Samsung Galaxy S4 i9505, Samsung Galaxy S6 Fl..."
4,000e619d,"[Motorola Moto X Force, Samsung Galaxy S5, Mot..."


In [19]:
df_model['cantidad_vistos'] = df_model['model'].apply(len)
df_model['model'] = df_model['model'].apply(lambda x: ', '.join(map(str, x)))
df_model = df_model.fillna('')
df_model.head()

,person,model,cantidad_vistos
0,00091926,"iPhone 6 Plus, iPhone 6S, iPhone 6S, Motorola ...",372
1,000ba417,"Samsung Galaxy A3 2016, Samsung Galaxy Gran Pr...",153
2,000c79fe,"iPhone 7, iPhone 7, iPhone 7",3
3,000e4d9e,"Samsung Galaxy S4 i9505, Samsung Galaxy S6 Fla...",339
4,000e619d,"Motorola Moto X Force, Samsung Galaxy S5, Moto...",28


In [22]:
word_vectorizer = CountVectorizer(vocabulary = marcas)

tf_mat = word_vectorizer.fit_transform(df_model['model'])

tf_array = tf_mat.toarray()
tf_array.shape

(35495, 7)

In [23]:
feature_names = word_vectorizer.get_feature_names()

In [24]:
df_tfidf = pd.DataFrame(tf_array, columns=feature_names)
df_tfidf['person'] = df_model['person']
df_tfidf['cantidad_vistos'] = df_model['cantidad_vistos']

In [28]:
df_tfidf.head()

,iphone,samsung,lg,motorola,sony,asus,lenovo,person,cantidad_vistos
0,251,61,2,55,1,0,1,00091926,372
1,8,105,4,35,1,0,0,000ba417,153
2,3,0,0,0,0,0,0,000c79fe,3
3,21,300,1,2,15,0,0,000e4d9e,339
4,7,11,3,7,0,0,0,000e619d,28


In [25]:
df_top = df_model['cantidad_vistos'].to_frame()
df_top['person'] = df_model['person']

In [26]:
df_top.shape

(35495, 2)

In [27]:
df_top.head()

,cantidad_vistos,person
0,372,00091926
1,153,000ba417
2,3,000c79fe
3,339,000e4d9e
4,28,000e619d


## Diff de lo de caro 0.62

In [16]:
df_diff = pd.read_csv('diff.csv')
df_top = df_top.merge(df_diff , left_on='person', right_on='person', how='left')
df_top.shape

(37130, 5)

## Most common searched values in conversion persons count vector
TOP 10 CON RF = ['5s', '6s', 'galaxy', 'iphone', 'j5', 'j7', 'moto', 'plus', 's8', 'samsung']

In [75]:
person_conv = (df_month.loc[df['event']=='conversion'])['person'].drop_duplicates()
df_conv = df.loc[df['person'].isin(person_conv)]

In [76]:
df_search = df.loc[df['search_term'].notnull()]
df_search = df_search.groupby('person')['search_term'].apply(list).reset_index()

In [77]:
#drop nans
df_search.head()

,person,search_term
0,000c79fe,"[Iphone 7, Galaxy a8, Iphone 7, Galaxy s8, Iph..."
1,000e619d,"[samsung rosa, sansung j7, sansung j7, sansung..."
2,001001be,"[IPhone 6, 5s, IPhone 6, IPhone 6, 5s, IPho..."
3,001802e4,"[Aiphone 6s, Aiphone 6s]"
4,0019e639,"[ON 7, ON 7, ON 7, ON 7, ON 7, ON 7, ON 7, ON ..."


In [78]:
#df_all = df_labels.merge(df_search, left_on = 'person', right_on = 'person', how = 'left')
#df_all.head()

In [79]:
df_search['search_term'] = df_search['search_term'].apply(lambda x: ', '.join(map(str, x)))
df_search = df_search.fillna('')
df_search.shape

(12570, 2)

In [80]:
word_vectorizer = CountVectorizer(vocabulary=marcas)

tf_mat = word_vectorizer.fit_transform(df_search['search_term'])

tf_array = tf_mat.toarray()
tf_array.shape

(12570, 7)

In [81]:
doc = 0
feature_names = word_vectorizer.get_feature_names()

In [82]:
df_countvect = pd.DataFrame(tf_array, columns=feature_names)
df_countvect['person'] = df_search['person']
df_countvect.head()

,iphone,samsung,lg,motorola,sony,asus,lenovo,person
0,6,0,0,0,0,0,0,000c79fe
1,0,1,0,0,0,0,0,000e619d
2,12,0,0,0,0,0,0,001001be
3,0,0,0,0,0,0,0,001802e4
4,0,1,0,0,0,0,0,0019e639


In [83]:
df_countvect['person'] = df_search['person']
df_countvect.shape

(12570, 8)

In [84]:
df_top = df_top.merge(df_countvect, left_on='person', right_on='person', how='left')

In [86]:

df_top.head()

,person,viewed,searched,lead,conversion,iphone,samsung,lg,motorola,sony,asus,lenovo
0,00091926,18.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000ba417,51.000000,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000c79fe,3.000000,9.0,NaN,NaN,6.0,0.0,0.0,0.0,0.0,0.0,0.0
3,000e4d9e,37.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000e619d,7.000000,3.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## XGboost

In [87]:
df_train = df_labels.merge(df_countvect , left_on='person', right_on='person', how='left')

In [88]:
df_train.shape

(19414, 9)

In [89]:
df_train.head()

,person,label,iphone,samsung,lg,motorola,sony,asus,lenovo
0,0566e9c1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6ec7ee77,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,abe7a2fb,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34728364,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,87ed62de,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
X, y = df_train.iloc[:,2:],df_train.iloc[:,1]
X = X.fillna(0)
X.head()

,iphone,samsung,lg,motorola,sony,asus,lenovo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [92]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [93]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 4,
                subsample = 0.8,
                gamma = 1,
                n_estimators = 15) #150 = 0.61730  #1500 = 0.61296 , 

In [94]:
xg_reg.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=15,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [95]:
persons = df_labels['person']
df_predict = df_top.loc[~df_top.person.isin(persons)]
ppl_to_predict = (df.loc[~df['person'].isin(persons)])['person'].to_frame()
ppl_to_predict = ppl_to_predict.drop_duplicates('person')
df_predict = ppl_to_predict.merge(df_predict, left_on = 'person' , right_on = 'person', how='left')
X_predict = df_predict.drop(['person'], axis=1)
X_predict.shape

(19415, 11)

In [96]:
entrie = xg_reg.predict(X_predict)

ValueError: feature_names mismatch: ['iphone', 'samsung', 'lg', 'motorola', 'sony', 'asus', 'lenovo'] ['viewed', 'searched', 'lead', 'conversion', 'iphone', 'samsung', 'lg', 'motorola', 'sony', 'asus', 'lenovo']
training data did not have the following fields: lead, searched, conversion, viewed

In [ ]:
seriesita = pd.Series(entrie)

In [97]:
df_entrie = df_predict['person'].to_frame()
df_entrie['label'] = seriesita

In [64]:
df_entrie.head()

,person,label
0,4886f805,0.140919
1,0297fc1e,0.145204
2,2d681dd8,0.145204
3,cccea85e,0.145204
4,4c8a8b93,0.145204


In [65]:
df_entrie = df_entrie.fillna(0)

In [66]:
num = df_entrie._get_numeric_data()
num[num < 0] = 0

In [67]:
df_entrie.to_csv(path_or_buf = 'entrie2.0', index = False)

In [68]:
df_entrie.shape

(19415, 2)

In [69]:
df_entrie['label'].nlargest()

2302    0.311732
2316    0.311732
4593    0.311732
4651    0.311732
4786    0.311732
Name: label, dtype: float32

## Scoring

Para evaluar usen esta medida que me da valores muy parecidos a los de kaggle, para hacer las predicciones usen el otro

In [98]:
my_classifier1 = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 6,
                subsample = 0.8,
                gamma = 1,
                n_estimators = 10)

In [99]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [100]:
my_classifier1.fit(X_train,y_train)
entrie = my_classifier1.predict_proba(X_predict)

ValueError: feature_names mismatch: ['iphone', 'samsung', 'lg', 'motorola', 'sony', 'asus', 'lenovo'] ['viewed', 'searched', 'lead', 'conversion', 'iphone', 'samsung', 'lg', 'motorola', 'sony', 'asus', 'lenovo']
training data did not have the following fields: lead, searched, conversion, viewed

In [101]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  my_classifier1.predict_proba(X_test)[:,1])

0.529419063717788

 # Random Forest feature importance
    -TEST THIS

In [115]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
#Load boston housing dataset as an example

names = X.columns
rf = RandomForestRegressor()
rf.fit(X, y)
print ("Features sorted by their score:")
zipped = zip(map(lambda x: round(x, 2), rf.feature_importances_), names)
feature = sorted(zipped, key=lambda x: x[1])

/home/amaherok/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Features sorted by their score:


In [116]:
X.shape

(19414, 6)

In [120]:
feat_importance = pd.DataFrame(feature, columns=['importance', 'feature'])
feat_importance

,importance,feature
0,0.1831,cantidad_vistos
1,0.0153,conversion
2,0.5295,diff
3,0.0072,lead
4,0.1036,searched
5,0.1613,viewed


In [1]:
feat_importance =feat_importance.loc[feat_importance['importance'].isin(feat_importance['importance'].nlargest(22))]
feats_servibles = feat_importance['feature'].tolist()
feats_servibles

NameError: name 'feat_importance' is not defined

In [119]:
df_top = df_top[feats_servibles]
df_top['person'] = subjects
df_top.head()

,cantidad_vistos,conversion,diff,lead,searched,viewed,person
0,4.0,NaN,0.058056,NaN,1.0,4.0,4886f805
1,20.0,NaN,0.095324,NaN,NaN,15.0,ad93850f
2,404.0,NaN,0.120095,1.0,2.0,26.0,0297fc1e
3,13.0,NaN,0.170093,NaN,1.0,7.0,2d681dd8
4,739.0,NaN,0.152070,NaN,1.0,94.0,cccea85e


In [121]:
feats_servibles

['cantidad_vistos', 'conversion', 'diff', 'lead', 'searched', 'viewed']

In [146]:
df_v = df.loc[df['event'] == 'viewed product']
df_v['timestamp'] = pd.to_datetime(df_v['timestamp'])
df_v['count'] = 1
df_v['month'] = df_v['timestamp'].dt.month
df_v['day'] = df_v['timestamp'].dt.day

df_v = df_v.groupby(['person','month', 'day','model']).agg({'count':'sum'})
#df_v = df_v.sort_values(by=['person','count'],ascending=False ).groupby('person').head(2).reset_index()

df_v = df_v.groupby(['person']).agg({'count':'mean'}).reset_index()
df_v.head()

,person,count
0,00091926,4.133333
1,00091a7a,1.000000
2,000ba417,4.371429
3,000c79fe,3.000000
4,000e4d9e,4.985294


In [147]:
df_s = df.loc[df['event'] == 'searched products']
df_s['timestamp'] = pd.to_datetime(df_s['timestamp'])
df_s['count'] = 1
df_s['month'] = df_s['timestamp'].dt.month
df_s['day'] = df_s['timestamp'].dt.day

df_s = df_s.groupby(['person','month', 'day', 'search_term']).agg({'count':'sum'})
#df_s = df_s.sort_values(by=['person','count'], ascending=False).groupby('person').head(2).reset_index()
df_s = df_s.groupby(['person']).agg({'count':'mean'}).reset_index()
df_s.head()

,person,count
0,000c79fe,1.142857
1,000e619d,3.000000
2,001001be,8.000000
3,001802e4,2.000000
4,0019e639,2.200000


In [148]:
df_s.columns = ['person','searched']
df_v.columns = ['person', 'viewed']
df_s.head()

,person,searched
0,000c79fe,1.142857
1,000e619d,3.000000
2,001001be,8.000000
3,001802e4,2.000000
4,0019e639,2.200000


In [149]:
df_vs_feat = df_v.merge(df_s,  on='person', how='left')
df_vs_feat.head()

,person,viewed,searched
0,00091926,4.133333,NaN
1,00091a7a,1.000000,NaN
2,000ba417,4.371429,NaN
3,000c79fe,3.000000,1.142857
4,000e4d9e,4.985294,NaN


In [270]:
df_top = df_top.merge(df_vs_feat, left_on='person', right_on='person', how='left')
df_top.shape

(38829, 3)